In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [2]:
transform_normal = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor()
])

transform_train = transforms.Compose([
    transforms.Resize((250, 250)),
    # Randomly change the brightness of the image
    transforms.ColorJitter(brightness=0.5),
    # Randomly flip the image horizontally
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically
    transforms.RandomRotation(55),  # Randomly rotate the image by 45 degrees
    transforms.ToTensor(),
    # transforms.Normalize(mean = [0.0, 0.0, 0.0], std = [1.0, 1.0, 1.0]) # Takes each value for the channel, subtracts the mean and divides by the standard deviation (value - mean) / std
])

# Define the transformations
transformations1 = transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((250, 250))])

# Load the dataset
training_dataset = torchvision.datasets.Flowers102(root='./data', split="train",
                                                   download=True, transform=transformations1)
testing_dataset = torchvision.datasets.Flowers102(root='./data', split="test",
                                                  download=True, transform=transformations1)
validation_dataset = torchvision.datasets.Flowers102(root='./data', split="val",
                                                     download=True, transform=transformations1)

# Create the dataloaders
train_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(testing_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(
    validation_dataset, batch_size=32, shuffle=False)

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.flatten = nn.Flatten()
        self.relu_func = nn.ReLU()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.25)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.25)
        )
        self.fc1 = nn.Linear(64 * 29 * 29, 1024)
        self.drop = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(1024, 102)

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        # Adjusted to match the output size
        self.fc1 = nn.Linear(64 * 29 * 29, 102)
        self.fc2 = nn.Linear(512, 256)  # Additional fully connected layer
        self.fc3 = nn.Linear(256, 102)  # Output layer for 102 classes

    def forward(self, x):
        x = self.pool(self.relu_func(self.conv1(x)))
        x = self.pool(self.relu_func(self.conv2(x)))
        x = self.pool(self.relu_func(self.conv3(x)))
        # x = x.view(-1, 64 * 29 * 29)  # Flatten the tensor
        x = self.flatten(x)
        x = self.fc1(x)
        return x

In [4]:
device = torch.device("mps")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [5]:
def NetworkAccuracyOnValidation():
    with torch.no_grad():
        num_class_correct = [0 for i in range(102)]
        num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predictions == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predictions[i]
                if label == pred:
                    num_class_correct[label] += 1
                num_class_samples[label] += 1

        acc = 100.0 * total_correct / total_samples
        print(f'Accuracy on validation set: {acc} %')
        return acc


def NetworkAccuracyOnTesting():
    with torch.no_grad():
        num_class_correct = [0 for i in range(102)]
        num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predictions == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predictions[i]
                if (label == pred):
                    num_class_correct[label] += 1
                num_class_samples[label] += 1

        acc = 100.0 * total_correct / total_samples
        print(f'Accuracy on testing set: {acc} %')

        for i in range(102):
            acc = 100.0 * num_class_correct[i] / num_class_samples[i]
            print(f'Accuracy of {i} : {acc} %')

        return acc

In [6]:
best_accuracy_epoch = 0
best_accuracy = 0

for epoch in range(60):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        images = images.to(device)
        labels = labels.to(device)
        
        label_pred = model(images)
        loss = criterion(label_pred, labels)
        
        predicted = torch.max(label_pred, 1)[1]
        batch_corr = (predicted == labels).sum()
        batch_acc = batch_corr.item() / len(images)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch Number {epoch}, Index = {i}/{len(train_loader)-1}, Loss = {loss.item()}")
    
    current_accuracy = NetworkAccuracyOnValidation()
    if (current_accuracy > best_accuracy):
        best_accuracy = current_accuracy
        best_accuracy_epoch = epoch
        
print(f"Best accuracy on validation split: {best_accuracy} at epoch {best_accuracy_epoch}")
model.eval()
NetworkAccuracyOnTesting()


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (32x215296 and 36864x1024)